In [1]:
###-----------------
### Import Libraries
###-----------------

import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss,accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler 
from sklearn.pipeline import Pipeline

import warnings
warnings.filterwarnings('ignore')

In [2]:
vehicle=pd.read_csv('Vehicle.csv')

vehicle.head()

,Comp,Circ,D.Circ,Rad.Ra,Pr.Axis.Ra,Max.L.Ra,Scat.Ra,Elong,Pr.Axis.Rect,Max.L.Rect,Sc.Var.Maxis,Sc.Var.maxis,Ra.Gyr,Skew.Maxis,Skew.maxis,Kurt.maxis,Kurt.Maxis,Holl.Ra,Class
0,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
1,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
2,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
3,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
4,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus


In [3]:
lbl=LabelEncoder()
vehicle['Class']=lbl.fit_transform(vehicle['Class'])
X=vehicle.drop('Class',axis=1)
y=vehicle['Class']

In [4]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=23,stratify=y)

In [5]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [6]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)

In [7]:
#logloss

lr=LogisticRegression()
params={'penalty':["l1","l2","elastic",None],
       'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','sag','saga']}
#Default scoring-acc score
gcv=GridSearchCV(lr,param_grid=params,cv=kfold,scoring="neg_log_loss")
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'penalty': 'l2', 'solver': 'newton-cg'}
-0.43019987069217275


In [8]:
kfold=StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
scalar=StandardScaler()
knr=KNeighborsClassifier() 
 
pipe=Pipeline([("SCL",scalar),("KNR",knr)])          #here we are using values of KNN transformer
params={'KNR__n_neighbors':np.arange(1,11),'SCL':[StandardScaler(),MinMaxScaler()]}

#log_loss
gcv=GridSearchCV(pipe,param_grid=params,cv=kfold,scoring='neg_log_loss')
gcv.fit(X,y)
print(gcv.best_params_)
print(gcv.best_score_)

{'KNR__n_neighbors': 10, 'SCL': StandardScaler()}
-1.0711613659010997


In [9]:
kfold = StratifiedKFold(n_splits= 5, shuffle= True, random_state= 23)
pipe = Pipeline([('scl', MinMaxScaler()),('knn', KNeighborsClassifier())])
params = {'scl': [StandardScaler(), MinMaxScaler()], 'knn__n_neighbors': np.arange(1,11)}

gscv = GridSearchCV(pipe, param_grid= params, cv = kfold, scoring='neg_log_loss')
gscv.fit(X,y)

print(gscv.best_estimator_)
print(gscv.best_score_)

Pipeline(steps=[('scl', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=10))])
-1.0711613659010997
